# Add various kinds of noise then test denoising methods

In [2]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def load_image_keep_channels(path):
    # preserve number of channels and alpha if present
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(path)
    # convert 4-channel BGRA -> BGR (drop alpha) for metric comparisons
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    return img

def to_uint8(img):
    # convert floats to uint8 if needed (assume values in 0..1 for floats)
    if img.dtype == np.float32 or img.dtype == np.float64:
        img = np.clip(img, 0.0, 1.0)
        img = (img * 255).astype(np.uint8)
    else:
        img = img.astype(np.uint8)
    return img

def compute_metrics(ref_path, test_path):
    A = load_image_keep_channels(ref_path)
    B = load_image_keep_channels(test_path)

    # If either is grayscale image read as 2D, keep as 2D. If one is 2D and the other 3D,
    # convert 3D to grayscale to compare apples-to-apples (or replicate channels)
    if A.ndim == 2 and B.ndim == 3:
        B = cv2.cvtColor(B, cv2.COLOR_BGR2GRAY)
    elif A.ndim == 3 and B.ndim == 2:
        A = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)

    A = to_uint8(A)
    B = to_uint8(B)

    if A.shape != B.shape:
        raise ValueError(f"Shape mismatch: {A.shape} vs {B.shape}")

    # data_range for uint8 images
    data_range = 255 if A.dtype == np.uint8 else (A.max() - A.min())

    # PSNR works for both grayscale and color
    psnr_val = psnr(A, B, data_range=data_range)

    # For SSIM: specify channel_axis for multichannel arrays
    if A.ndim == 3:
        ssim_val = ssim(A, B, data_range=data_range, channel_axis=-1)
    else:
        ssim_val = ssim(A, B, data_range=data_range)

    return psnr_val, ssim_val

In [3]:
def AWGN(image, mean=0, sigma=25):
    """Add Additive White Gaussian Noise to an image (uint8 0–255)."""
    image = image.astype(np.float32)
    gauss = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def impulse_noise(image, prob=0.05):
    """Add Impulse Noise (Salt and Pepper Noise) to an image."""
    noisy_image = image.copy()
    black = 0
    white = 255
    probs = np.random.rand(*image.shape)
    noisy_image[probs < (prob / 2)] = black
    noisy_image[probs > 1 - (prob / 2)] = white
    return noisy_image

In [4]:
import os
import matplotlib.pyplot as plt
# add noise to image and save them to respective folders (AWGN and Impulse Noise)
def add_noise_and_save(input_image_path, awgn_folder, impulse_folder):
    if not os.path.exists(awgn_folder):
        os.makedirs(awgn_folder)
    if not os.path.exists(impulse_folder):
        os.makedirs(impulse_folder)

    image = load_image_keep_channels(input_image_path)

    awgn_image = AWGN(image)
    impulse_image = impulse_noise(image)

    awgn_image_path = os.path.join(awgn_folder, os.path.basename(input_image_path))
    impulse_image_path = os.path.join(impulse_folder, os.path.basename(input_image_path))

    cv2.imwrite(awgn_image_path, awgn_image)
    cv2.imwrite(impulse_image_path, impulse_image)

    return awgn_image_path, impulse_image_path



In [5]:
# add noise to all the .bmp images in STI/Classic
input_folder = 'C:\\ImageProcessing\\Project_5\\original'
awgn_folder = 'C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN'
impulse_folder = 'C:\\ImageProcessing\\Project_5\\noisy_images\\Impulse'
for filename in os.listdir(input_folder):
    if filename.endswith('.bmp'):
        input_image_path = os.path.join(input_folder, filename)
        add_noise_and_save(input_image_path, awgn_folder, impulse_folder)

#calculate PSNR and SSIM for noisy images
results = []
for filename in os.listdir(input_folder):
    if filename.endswith('.bmp'):
        input_image_path = os.path.join(input_folder, filename)
        awgn_image_path = os.path.join(awgn_folder, filename)
        impulse_image_path = os.path.join(impulse_folder, filename)

        psnr_awgn, ssim_awgn = compute_metrics(input_image_path, awgn_image_path)
        psnr_impulse, ssim_impulse = compute_metrics(input_image_path, impulse_image_path)

        results.append((filename, psnr_awgn, ssim_awgn, psnr_impulse, ssim_impulse))
# Print results in tabular format
print(f"{'Image':<20} {'PSNR_AWGN':<15} {'SSIM_AWGN':<15} {'PSNR_Impulse':<15} {'SSIM_Impulse':<15}")
for row in results:
    print(f"{row[0]:<20} {row[1]:<15.4f} {row[2]:<15.4f} {row[3]:<15.4f} {row[4]:<15.4f}")
# Create CSV 
import csv
csv_file = 'C:\\ImageProcessing\\Project_5\\noise_metrics_results.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'PSNR_AWGN', 'SSIM_AWGN', 'PSNR_Impulse', 'SSIM_Impulse'])
    for row in results:
        writer.writerow(row)
        
# Visualize one example
example_image = os.path.join(input_folder, 'boatsColor.bmp')
awgn_image_path = os.path.join(awgn_folder, 'boatsColor.bmp')
impulse_image_path = os.path.join(impulse_folder, 'boatsColor.bmp')
original = load_image_keep_channels(example_image)
awgn_image = load_image_keep_channels(awgn_image_path)
impulse_image = load_image_keep_channels(impulse_image_path)
cv2.imshow('Original Image', original)
cv2.imshow('AWGN Image', awgn_image)
cv2.imshow('Impulse Noise Image', impulse_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image                PSNR_AWGN       SSIM_AWGN       PSNR_Impulse    SSIM_Impulse   
airplane.bmp         20.3604         0.2925          17.8673         0.3503         
boats.bmp            20.3544         0.3188          18.4257         0.3495         
BoatsColor.bmp       20.5031         0.3356          18.1556         0.3656         
checkerboard.bmp     23.1708         0.2956          16.0446         0.4280         
goldhill.bmp         20.4518         0.3399          18.0012         0.3768         


In [6]:
boats_path = 'C:\\ImageProcessing\\Project_5\\original\\boats.bmp'
for i in range(1, 30, 5):
    noisy_image = AWGN(load_image_keep_channels(boats_path), sigma=i)
    noisy_image_path = f'C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN\\boats_sigma_{i}.bmp'
    cv2.imwrite(noisy_image_path, noisy_image)
    psnr_val, ssim_val = compute_metrics(boats_path, noisy_image_path)
    print(f'Sigma: {i}, PSNR: {psnr_val:.2f}, SSIM: {ssim_val:.4f}')
    

Sigma: 1, PSNR: 46.88, SSIM: 0.9913
Sigma: 6, PSNR: 32.53, SSIM: 0.8002
Sigma: 11, PSNR: 27.34, SSIM: 0.5983
Sigma: 16, PSNR: 24.15, SSIM: 0.4641
Sigma: 21, PSNR: 21.84, SSIM: 0.3726
Sigma: 26, PSNR: 20.04, SSIM: 0.3077


In [7]:
# import cv2

# noisy_path = "C:\\ImageProcessing\\Project_5\\noisy_images\\Impulse\\checkerboard.bmp"
# den_path = "C:\\ImageProcessing\\Project_5\\Checker_ChatGPT.png"
# orig_path = "C:\\ImageProcessing\\Project_5\\original\\checkerboard.bmp" 

# orig = cv2.imread(orig_path)      # or .bmp, .jpg, etc.
# den = cv2.imread(den_path)
# noisy = cv2.imread(noisy_path)

# h, w = orig.shape[:2]
# den_resized = cv2.resize(den, (w, h), interpolation=cv2.INTER_CUBIC)
# cv2.imwrite("denoised_match.bmp", den_resized)

In [ ]:
import deepinv as dinv
import torch
import cv2
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Pretrained DRUNet from deepinv (downloads weights on first use)
drunet = dinv.models.DRUNet(pretrained="download", device=device).eval()
drunet_greyscale = dinv.models.DRUNet(pretrained="download", device=device, in_channels=1, out_channels=1).eval()


def DRUNet_denoise(noisy_image, sigma=0.1, device=device):
    """
    Denoise a noisy image using DRUNet.

    Parameters
    ----------
    noisy_image : np.ndarray
        HxW (gray) or HxWx3 (color), usually uint8 in [0,255] or float in [0,1].
    sigma : float
        AWGN std *in [0,1]* matching how you added noise.
        e.g. AWGN with sigma=25 on uint8 -> sigma = 25/255 ≈ 0.098.
    """

    # determine if grayscale or color
    if noisy_image.ndim == 2:
        # gray -> 1-channel so DRUNet (greyscale) can process it
        noisy_image = noisy_image[:, :, np.newaxis]
        drunet_used = drunet_greyscale
    elif noisy_image.ndim == 3 and noisy_image.shape[2] == 1:
        noisy_image = noisy_image.repeat(3, axis=2)
        drunet_used = drunet_greyscale
    else:
        drunet_used = drunet
    

    # If BGR from OpenCV, convert to RGB (DRUNet was trained on RGB)
    noisy_rgb = noisy_image
    if noisy_rgb.dtype == np.uint8:
        noisy_rgb = noisy_rgb.astype(np.float32) / 255.0

    # ----- 2) Numpy -> torch tensor, shape (B,C,H,W) -----
    x = torch.from_numpy(noisy_rgb).permute(2, 0, 1).unsqueeze(0).to(device)

    # ----- 3) Run DRUNet -----
    with torch.no_grad():
        # DRUNet expects sigma in [0,1] for images in [0,1]
        x_denoised = drunet_used(x, sigma=sigma)

    # ----- 4) Back to numpy -----
    denoised = x_denoised.squeeze(0).permute(1, 2, 0).cpu().numpy()
    denoised = np.clip(denoised, 0.0, 1.0)

    # Back to uint8 BGR for OpenCV display/saving
    denoised_uint8 = (denoised * 255.0).round().astype(np.uint8)

    # convert back to BGR if you want consistent OpenCV handling
    denoised_bgr = denoised_uint8  # if your input was RGB; if it was BGR, swap here

    return denoised_bgr




Downloading: "https://huggingface.co/deepinv/drunet/resolve/main/drunet_deepinv_gray_finetune_26k.pth?download=true" to C:\Users\scout/.cache\torch\hub\checkpoints\drunet_deepinv_gray_finetune_26k.pth
100%|██████████| 125M/125M [00:01<00:00, 92.4MB/s] 


In [ ]:

# find best sigma norm for DRUNet on AWGN noisy BoatsColor.bmp
best_psnr = -np.inf
best_sigma = None
for sigma in range(20, 30, 1):
    sigma_norm = sigma / 255.0
    img = cv2.imread("C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN\\BoatsColor.bmp")
    denoised = DRUNet_denoise(img, sigma=sigma_norm, device=device)
    cv2.imwrite("temp_denoised.bmp", denoised)
    psnr_value, ssim_value = compute_metrics("C:\\ImageProcessing\\Project_5\\original\\BoatsColor.bmp", "temp_denoised.bmp")
    print(f"Sigma: {sigma}, PSNR: {psnr_value:.2f}, SSIM: {ssim_value:.4f}")
    if psnr_value > best_psnr:
        best_psnr = psnr_value
        best_sigma = sigma_norm
# cv2.imshow("Noisy", img)
# cv2.imshow("DRUNet Denoised", denoised)
# cv2.imshow("Original", cv2.imread("C:\\ImageProcessing\\Project_5\\original\\BoatsColor.bmp"))
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Sigma: 20, PSNR: 29.27, SSIM: 0.6973
Sigma: 21, PSNR: 30.30, SSIM: 0.7475
Sigma: 22, PSNR: 31.30, SSIM: 0.7966
Sigma: 23, PSNR: 32.16, SSIM: 0.8387
Sigma: 24, PSNR: 32.71, SSIM: 0.8693
Sigma: 25, PSNR: 32.81, SSIM: 0.8753
Sigma: 26, PSNR: 32.73, SSIM: 0.8733
Sigma: 27, PSNR: 32.63, SSIM: 0.8710
Sigma: 28, PSNR: 32.53, SSIM: 0.8686
Sigma: 29, PSNR: 32.42, SSIM: 0.8662


In [ ]:
# denoise all images in AWGN folder and save results
awgn_folder = 'C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN'
denoised_folder = 'C:\\ImageProcessing\\Project_5\\denoised\\DRUNet(adaptive_channels)'
if not os.path.exists(denoised_folder):
    os.makedirs(denoised_folder)
for filename in os.listdir(awgn_folder):
    if filename.endswith('.bmp'):
        noisy_image_path = os.path.join(awgn_folder, filename)
        img = cv2.imread(noisy_image_path)
        denoised = DRUNet_denoise(img, sigma=sigma_norm, device=device)
        denoised_image_path = os.path.join(denoised_folder, filename)
        cv2.imwrite(denoised_image_path, denoised)
#!/usr/bin/env python3

In [11]:
# calculate PSNR and SSIM for denoised images and save as csv
results = []
denoised_folder = 'C:\\ImageProcessing\\Project_5\\denoised\\DnCNN'
for filename in os.listdir(denoised_folder):
    if filename.endswith('.bmp'):
        denoised_image_path = os.path.join(denoised_folder, filename)
        original_image_path = os.path.join(input_folder, filename)
        psnr_val, ssim_val = compute_metrics(original_image_path, denoised_image_path)
        results.append((filename, psnr_val, ssim_val))
# CSV
import csv
csv_path = 'C:\\ImageProcessing\\Project_5\\denoised\\DnCNN\\denoising_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'PSNR', 'SSIM'])
    for row in results:
        writer.writerow(row)

